In [17]:
import pandas as pd
import pickle
import os
import glob
import re
import string
import emoji
import numpy as np
import matplotlib.pyplot as plt

import twint
import nest_asyncio
nest_asyncio.apply()

from bokeh.plotting import figure, output_file, save
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Legend, HoverTool
from bokeh.embed import components
output_notebook()

from bs4 import BeautifulSoup

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

Loading BokehJS ...

In [18]:
with open('data/word_matrices/nyc_mayor.pkl', 'rb') as file:
    nyc_mayor_word_matrix = pickle.load(file)

In [19]:
def scrape_user(username):
    if username[0] == '@':
        username = username[1:]
    
    c = twint.Config()
    c.Hide_output = True
    c.Username = username
    c.Pandas = True
    c.Limit = 500
    c.Filter_retweets = False
    
    twint.run.Search(c)
    
    tweets_df = twint.storage.panda.Tweets_df
    
    columns_wanted = ['date', 'tweet']
    tweets_df_filtered = tweets_df[columns_wanted]
        
    return tweets_df_filtered

def clean_tweet_sentiment_analysis(tweet):
    
    #converts html to text
    tweet = BeautifulSoup(tweet, 'lxml').text
    
    #removes links
    tweet = re.sub(r'http\S+', '', tweet)
    
    #removes twitter usernames
    tweet = re.sub(r'(\s)?@\w+', '', tweet)
    
    return(tweet)

def calc_tweet_sentiment(tweet):    
    sentiment_analyzer = SentimentIntensityAnalyzer()
    return sentiment_analyzer.polarity_scores(tweet)['compound']

def clean_tweet_tfidf(tweet):
    
    tweet = tweet.lower()
    
    #removing punctuation
    cleaned_tweet = re.sub('[%s]' % re.escape(custom_punctuation), '', tweet)
    
    #removing emojis
    cleaned_tweet = re.sub(emoji.get_emoji_regexp(), r"", cleaned_tweet)
        
    #remove stop words
    cleaned_tweet = ' '.join([item for item in cleaned_tweet.split() if item not in custom_stop_words])
        
    return(cleaned_tweet)

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_tweets(tweets):
    #initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    #lemmatize the words
    return [' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(tweet)]) for tweet in tweets]

def calc_word_vectors(df, words):        
    word_scores = []
    for word in words:
        word_df = df[df['lemmatized_tweets'].str.contains(word)]
        if word_df.empty:
            word_scores.append(0)
        else:
            word_scores.append(word_df['sentiment'].mean())
    return(word_scores)

def create_similarity_matrix(similarity_tool, df_vectors):
        
    similarities = np.zeros((len(df_vectors), len(df_vectors)))
    
    for i in range(len(df_vectors)):
        politician_1 = df_vectors.iloc[i,:].values.reshape(1, -1)
        for j in range(i, len(df_vectors)):
            politician_2 = df_vectors.iloc[j, :].values.reshape(1, -1)
            similarities[i][j] = similarities[j][i] = similarity_tool(politician_1,politician_2)
                
    df = pd.DataFrame(similarities, index = df_vectors.index, columns = df_vectors.index)      
    
    return df

def determine_user_similarity(username, reference_matrix):
    matrix_copy = reference_matrix.copy()
    
    #scrape profile of entered user
    user_df = scrape_user(username)
    
    #clean tweets for sentiment analysis
    user_df['tweet'] = user_df['tweet'].apply(lambda x: clean_tweet_sentiment_analysis(x))
    
    #perform sentiment analysis on user's tweets
    user_df['sentiment'] = user_df['tweet'].apply(lambda x: calc_tweet_sentiment(x))
    
    #prepare tweets for tf-idf vectorization
    user_df['cleaned_tweets'] = user_df['tweet'].apply(lambda x: clean_tweet_tfidf(x))
    user_df['lemmatized_tweets'] = lemmatize_tweets(user_df['cleaned_tweets'])
    
    #determine word vectors for entered user
    user_word_vector = calc_word_vectors(user_df, reference_matrix.columns.to_list())
    
    matrix_copy.loc[username] = user_word_vector
        
    cos_sim_df = create_similarity_matrix(cosine_similarity, matrix_copy)
    
    similarity_vector = cos_sim_df[username].sort_values(ascending=False).iloc[1:].to_frame()
    
    return(similarity_vector)

In [20]:
%%time

custom_stop_words = [word.replace("'", "") for word in stopwords.words('english')] + ['rt', 'amp', 'u', 'w', 'im', 'live', 'must', 'join', 'tune', 'pm', 'et', 
                         'year', 'say', 'get', 'it']
custom_punctuation = '!"#&\'()*+,-./:;<=>?@[\\]^_`{|}~\''
    
a = determine_user_similarity('joebiden', nyc_mayor_word_matrix)

print(a.sort_values)

/Users/msternke/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/msternke/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/msternke/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

<bound method DataFrame.sort_values of                            joebiden
Andrew-Yang                0.508831
Carlos-Menchaca            0.492485
Barbara-Kavovit            0.482701
Joycelyn-Taylor            0.480289
Dianne-Morales             0.470763
Christopher-S.-Krietchman  0.462382
Eric-Adams                 0.454084
Scott-Stringer             0.447400
Paperboy-Prince            0.434849
Maya-Wiley                 0.412642
Cleopatra-Fitzgerald       0.380909
Abbey-Laurel-Smith         0.347860
Stacey-Prussman            0.300517
Bill-Pepitone              0.083131>
CPU times: user 8.61 s, sys: 365 ms, total: 8.98 s
Wall time: 13.2 s


In [22]:
def make_sim_bar_chart(matrix):
    
    p = figure(x_range=matrix.index.to_list(), plot_height=400,
               plot_width=600, toolbar_location=None)

    p.vbar(x=[i.replace("-"," ") for i in matrix.index], top=matrix.values.flatten(), width=0.9)

    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = 'Similarity'
    p.xaxis.major_label_orientation = 45
    p.xaxis.major_label_text_font_size = "12pt"
    p.axis.axis_label_text_font_size = "12pt"
    p.add_tools(HoverTool(tooltips=[("Politician", "@x"), ("Similarity", "@top")]))
    
    return(p)

fig = make_sim_bar_chart(a)
show(fig)

In [52]:
def cos_sim(n1, n2):
    return np.dot([n1], [n2]) / (n1**2 * n2**2)

print(cos_sim(1, -3))

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'